In [10]:
!pip install slicer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!apt install DICOMLib

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package DICOMLib


In [ ]:
import slicer
volume = slicer.util.loadVolume('C:/Users/shamb/Downloads/110122_CE_2ch.nii.gz')
dicomWrite = slicer.modules.createdicomseries

In [11]:
import slicer
#from DICOMLib import DICOMExportScalarVolume
volume = slicer.util.loadVolume('/content/spleen_2.nii.gz')
DICOMExportScalarVolume(volumeNode = volume, studyUID = None, tags = None, directory = '/content/dcom_file')

AttributeError: ignored



---



---



In [12]:
!pip install pydicom nibabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00


In [14]:
import nibabel as nib

nii_img = nib.load('/content/spleen_2.nii.gz')

In [15]:
nii_array = nii_img.get_fdata()
nii_affine = nii_img.affine
nii_header = nii_img.header

In [20]:
print(nii_header)


<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 512 512  90   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [1.       0.794922 0.794922 5.       0.       0.       0.       0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'5.0.10'
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -406.20514
qoffset_y       : -406.20514
qoffset_z 

In [17]:
import pydicom
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid
import os

# set the DICOM tags for the series
series_meta = Dataset()
series_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2' # CT Image Storage
series_meta.MediaStorageSOPInstanceUID = generate_uid()

series_ds = FileDataset('output_file.dcm', {}, file_meta=series_meta, preamble=b"\0" * 128)

series_ds.PatientName = "Test^Patient"
series_ds.PatientID = "12345"
series_ds.Modality = "CT"
series_ds.SeriesInstanceUID = generate_uid()
series_ds.StudyInstanceUID = generate_uid()

# create a new file dataset for each slice
for i in range(nii_array.shape[2]):
    # set the DICOM tags for the file
    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2' # CT Image Storage
    file_meta.MediaStorageSOPInstanceUID = generate_uid()

    ds = Dataset()

    ds.PatientName = "Test^Patient"
    ds.PatientID = "12345"
    ds.Modality = "CT"
    ds.SeriesInstanceUID = series_ds.SeriesInstanceUID
    ds.StudyInstanceUID = series_ds.StudyInstanceUID
    ds.FrameOfReferenceUID = generate_uid()
    ds.SOPInstanceUID = generate_uid()

    # set the image dimensions and pixel data
    ds.Rows, ds.Columns = nii_array.shape[0], nii_array.shape[1]
    ds.PixelSpacing = [1.0, 1.0]
    ds.ImageOrientationPatient = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
    ds.SliceThickness = nii_header['pixdim'][3]
    ds.SliceLocation = nii_affine[2, 3] + i * nii_header['pixdim'][3]
    ds.ImagePositionPatient = [nii_affine[0, 3], nii_affine[1, 3], ds.SliceLocation]
    ds.PixelData = nii_array[:, :, i].tobytes()

    # add the DICOM tags to the dataset
    ds.add_new((0x0008, 0x0008), 'CS', "ORIGINAL\\PRIMARY")
    ds.add_new((0x0008, 0x0012), 'DA', series_ds.StudyDate)
    ds.add_new((0x0008, 0x0013), 'TM', series_ds.StudyTime)
    ds.add_new((0x0008, 0x0020), 'DA', series_ds.StudyDate)
    ds.add_new((0x0008, 0x0030), 'TM', series_ds.StudyTime)
    ds.add_new((0x0020, 0x0013), 'IS', str(i+1).zfill(3))

    # add the file dataset to the series dataset
    series_ds.add(ds)

# save the series dataset
#series_ds.save_as('output_file.dcm')

series_ds.save_as('/content/dcom_file/output_file.dcm')


AttributeError: ignored